In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from numpy.random import randn
from scipy import stats
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.tools.plotting import radviz

Cuando tengas el dataframe sp con la columna 'Sector' rellena, hay que hacer un indice de con el comportamiento de cada sector, para luego graficar los 13 sectores en base 100.
Con la info del dataframe:

In [2]:
#----------- Download and print constituents-financials.csv file from GitHub ----------- 
# https://github.com/datasets/s-and-p-500-companies/blob/master/data/constituents-financials.csv 
pwd = os.getcwd()
path1 = pwd+'\\'+'constituents-financials.csv'
consti = pd.read_csv(path1)
#print(consti)
cn = pd.DataFrame(consti.loc[:,['Symbol','Sector']])
print (cn.head(10))

  Symbol                  Sector
0    MMM             Industrials
1    ABT             Health Care
2   ABBV             Health Care
3    ACN  Information Technology
4   ATVI  Information Technology
5    AYI             Industrials
6   ADBE  Information Technology
7    AAP  Consumer Discretionary
8    AES               Utilities
9    AET             Health Care


In [7]:
path ='C:\\Users\\ffuen\\Desktop\\all_stocks_5yr.csv'
sp500 = pd.read_csv(path , delimiter=',',decimal='.', header=0)
sp = pd.DataFrame(sp500)
sp['Sector'] = 100

In [4]:
sectors = np.unique(cn['Sector'])
indicesector = pd.DataFrame(data=None, columns = sectors)
indicesector['Date'] = pd.unique(sp.Date)
indicesector = indicesector.set_index('Date')
indicesector.iloc[0][:] = 100
print(indicesector.head(5))

           Consumer Discretionary Consumer Staples Energy Financials  \
Date                                                                   
2012-08-13                    100              100    100        100   
2012-08-14                    NaN              NaN    NaN        NaN   
2012-08-15                    NaN              NaN    NaN        NaN   
2012-08-16                    NaN              NaN    NaN        NaN   
2012-08-17                    NaN              NaN    NaN        NaN   

           Health Care Industrials Information Technology Materials  \
Date                                                                  
2012-08-13         100         100                    100       100   
2012-08-14         NaN         NaN                    NaN       NaN   
2012-08-15         NaN         NaN                    NaN       NaN   
2012-08-16         NaN         NaN                    NaN       NaN   
2012-08-17         NaN         NaN                    NaN       NaN  

Una vez construido el dataframe rellenar las celdas con las variaciones diarias del índice sectorial.

El índice se construye:

In [8]:
# Ordenamos el dataframe por fecha y luego por valores.
sectorchange = sp.groupby(['Sector','Name','Date'])['Close'].sum()

# Desapilamos, y reordenamos el dataframe: filas son fechas, y columnas ordenadas por 4 grupos (Open, High, Low y Close),
# y cada uno de los 4 grupos tiene 503 valores.
sectorchange = sectorchange.unstack(level=1).pct_change()+1

# Transponemos la matriz: columnas son fechas, y filas agrupadas por 4 grupos (Open, High, Low y Close), y cada uno de los 
# 4 grupos tiene 503 valores.
sectorchange = sectorchange.transpose()

# Recogemos la variación diaria media de cada columna y cada fecha, y le sumamos uno para multiplicar posteriormente el 
# valor inicial de la columna por este factor.
sectorchange = sectorchange.mean(level=0).transpose()
print(sectorchange)

Name                      A       AAL       AAP      AAPL      ABBV       ABC  \
Sector Date                                                                     
100    2012-08-13       NaN       NaN       NaN       NaN       NaN       NaN   
       2012-08-14  1.002724       NaN  1.022573  1.002667       NaN  0.992226   
       2012-08-15  0.999753       NaN  1.008915  0.998670       NaN  0.992949   
       2012-08-16  0.917737       NaN  1.016269  1.008766       NaN  0.988427   
       2012-08-17  1.001884       NaN  1.005658  1.018480       NaN  0.998936   
       2012-08-20  0.985492       NaN  0.982297  1.026245       NaN  1.004529   
       2012-08-21  1.012268       NaN  1.005868  0.986319       NaN  0.996022   
       2012-08-22  1.008888       NaN  1.002778  1.019526       NaN  1.009585   
       2012-08-23  1.000267       NaN  0.996676  0.990686       NaN  1.001582   
       2012-08-24  1.010141       NaN  1.000139  1.000951       NaN  1.003160   
       2012-08-27  0.989168 

[1258 rows x 503 columns]


In [ ]:
for i in sectorchange.columns:
    indicesector[i][1:] = indicesector[i][0] * np.cumprod(sectorchange[i][1:])
print(indicesector)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(16,8))
for sector in indicesector.columns:
    indicesector.loc[sector].plot(grid = True)
    plt.legend([sector], loc = 0)

plt.title ('Evolución diaria de cada sector')
plt.ylabel('Precios')
plt.legend(['Close'], loc = 0)